# 1. Namestitev knjižnic
Za delo s PostgreSQL v Pythonu potrebujemo knjižnico `psycopg2`. To knjižnico namestimo z uporabo ukaza `pip install psycopg2`.

# 2. Povezava z bazo PostgreSQL
Vzpostavimo povezavo z bazo PostgreSQL. Uporabimo ime baze, uporabniško ime, geslo, gostitelja (`localhost`) in vrata (`5432`).

In [1]:
import psycopg2

# Povezava z bazo PostgreSQL
conn = psycopg2.connect(
    dbname="pgvector_test",  # Ime baze
    user="test_user",        # Uporabniško ime PostgreSQL
    password="123",          # Geslo za PostgreSQL
    host="localhost",        # Gostitelj (npr. localhost)
    port="5432"              # Privzeta vrata PostgreSQL
)

# Ustvarimo kurzor za izvajanje poizvedb
cursor = conn.cursor()
print("Povezava z bazo je uspešna!")


ModuleNotFoundError: No module named 'psycopg2'

# 3. Ustvarjanje razširitve `pgvector`
Preverimo, ali je razširitev `pgvector` nameščena v bazi. Če razširitev še ne obstaja, jo ustvarimo z ukazom `CREATE EXTENSION`.

In [8]:
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
print("Razširitev pgvector je omogočena.")


Razširitev pgvector je omogočena.


# 4. Ustvarjanje tabele
Ustvarimo tabelo `products` z naslednjimi polji:
- `id`: Unikatni identifikator izdelka (avtomatsko se povečuje).
- `name`: Ime izdelka (besedilo).
- `embedding`: Vektorska predstavitev izdelka, ki vsebuje 3 vrednosti.


In [9]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    embedding VECTOR(3) -- Dolžina vektorja (3 za primer)
);
""")
conn.commit()
print("Tabela 'products' je uspešno ustvarjena.")


Tabela 'products' je uspešno ustvarjena.


# 5. Vstavljanje podatkov
Dodamo nekaj izdelkov z njihovimi vektorskimi predstavitvami v tabelo `products`.
Vsak vektor vsebuje 3 vrednosti, ki predstavljajo lastnosti izdelka.


In [10]:
data = [
    ('Jakna', [0.1, 0.2, 0.3]),
    ('Hlače', [0.2, 0.3, 0.4]),
    ('Majica', [0.3, 0.2, 0.1])
]

for name, embedding in data:
    cursor.execute("INSERT INTO products (name, embedding) VALUES (%s, %s)", (name, embedding))
conn.commit()
print("Podatki so uspešno vstavljeni.")


Podatki so uspešno vstavljeni.


# 6. Iskanje podobnih vektorjev
Poiščemo izdelek, ki je najbolj podoben določenemu vektorju `[0.25, 0.3, 0.35]`.
Uporabimo operator `<->`, ki izračuna razdaljo med vektorji.

In [11]:
search_vector = [0.25, 0.3, 0.35]
cursor.execute("""
SELECT name, embedding
FROM products
ORDER BY embedding <-> %s::vector
LIMIT 1;
""", (search_vector,))
result = cursor.fetchone()
print("Najbolj podoben izdelek:", result[0])

Najbolj podoben izdelek: Hlače


# 7. Posodobitev podatkov
Posodobimo vektorsko predstavitev za določen izdelek. V tem primeru posodobimo vektor izdelka `Jakna` na `[0.15, 0.25, 0.35]`.

In [12]:
cursor.execute("""
UPDATE products
SET embedding = %s::vector
WHERE name = %s;
""", ([0.15, 0.25, 0.35], 'Jakna'))
conn.commit()
print("Vektorska predstavitev izdelka 'Jakna' je posodobljena.")


Vektorska predstavitev izdelka 'Jakna' je posodobljena.


# 8. Brisanje podatkov
Iz tabele izbrišemo izdelek z imenom `Majica`.

In [ ]:
cursor.execute("""
DELETE FROM products
WHERE name = %s;
""", ('Majica',))
conn.commit()
print("Izdelek 'Majica' je izbrisan.")

# 9. Cosinusna podobnost
Cosinusna podobnost se pogosto uporablja za merjenje podobnosti med vektorji. Rezultat se giblje med -1 in 1:
- **1:** Popolna podobnost.
- **0:** Popolna neodvisnost.
- **-1:** Popolno nasprotje.

V tej tabeli računamo cosinusno podobnost med podanim vektorjem in vektorji v tabeli `products`.

In [19]:
search_vector = [0.25, 0.3, 0.35]
cursor.execute("""
SELECT name, (1 - (embedding <=> %s::vector)) AS similarity
FROM products
ORDER BY similarity DESC
LIMIT 3;
""", (search_vector,))
results = cursor.fetchall()

print("Top 3 najbolj podobni izdelki:")
for name, similarity in results:
    print(f"Izdelek: {name}, Cosinusna podobnost: {similarity:.4f}")

Top 3 najbolj podobni izdelki:
Izdelek: Hlače, Cosinusna podobnost: 0.9915
Izdelek: Jakna, Cosinusna podobnost: 0.9838
Izdelek: Majica, Cosinusna podobnost: 0.8664


# 10. Iskanje vektorjev z določeno razdaljo
Ta funkcionalnost omogoča iskanje vseh vektorjev, katerih razdalja od iskalnega vektorja je manjša od določene meje (npr. 0.1).
Uporabno za iskanje izdelkov, ki so dovolj podobni določenemu iskalnemu profilu.

In [20]:
search_vector = [0.2, 0.3, 0.4]
distance_threshold = 0.1

cursor.execute("""
SELECT name, embedding <-> %s::vector AS distance
FROM products
WHERE embedding <-> %s::vector < %s
ORDER BY distance ASC;
""", (search_vector, search_vector, distance_threshold))
results = cursor.fetchall()

print("Izdelki z razdaljo manj kot 0.1:")
for name, distance in results:
    print(f"Izdelek: {name}, Razdalja: {distance:.4f}")


Izdelki z razdaljo manj kot 0.1:
Izdelek: Hlače, Razdalja: 0.0000
Izdelek: Jakna, Razdalja: 0.0866
